In [4]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize, PunktSentenceTokenizer
from nltk.corpus import stopwords
#nltk.download()
import string
from nltk.stem import PorterStemmer
from nltk import pos_tag, ne_chunk
import nltk

In [5]:
#file with data
f = pd.read_csv(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\Airbnb_Texas_Rentals.csv')

In [6]:
#creating the .tsv files
for i in range(f.index.max()+1):
    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(i) + '.tsv', 'w', encoding="utf-8")
    for j in range(10):
        op.write('%s\t' %f.iloc[i, j])
    op.close()

### CREATING THE VOCABULARY

In [19]:
#FUNCTIONS
def preprocess(l):
    final = []
    for i in l:
        if not((ps.stem(i) in stopWords) or (ps.stem(i) in (string.punctuation) )):
            final.append(ps.stem(i))
    return (final)

def vocabularization(vocabulary, final, index):
    for word in final:
        if not(word in vocabulary):
            vocabulary[word] = index
            index = index + 1
    return(vocabulary, index)

#not used at the moment
def chunking(sentence):
    names = []
    l = []
    for chunk in ne_chunk(pos_tag(word_tokenize(sentence))) : 
        if type(chunk) is nltk.tree.Tree:
            s = ''
            for  i in chunk:
                s = s + ' ' + i[0]
            names.append(s[1:])
        else:
            l.append(chunk[0])
    return(names, l)

In [28]:
stopWords = set(stopwords.words('english'))
ps = PorterStemmer()
string.punctuation = string.punctuation + '–“”’'

vocabulary= {}
index = 0

for i in range(18259):
    
    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(i) + '.tsv', 'r', encoding="utf-8")
    for line in op:
        ou = line.strip().split('\t')
        sentence = ou[5].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ') + ' ' + ou[8]
    op.close()
    
    #splitting the sentence into chunks recognised through tags
    #names, l = chunking(sentence) 
    
    #preprocessing data deleting stop words, punctuations, ecc.  
    #final = preprocess(l) + names
    final = preprocess(word_tokenize(sentence))
    
    # IF  word not in vocabulary -> add the word
    vocabulary, index = vocabularization(vocabulary, final, index)
            
op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\vocabulary.txt', 'w', encoding="utf-8")
op.write(str(vocabulary))
op.close()

### INVERTED INDEX

In [31]:
inverted_index = {}

for file in range(18259):

    op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\doc\doc_' + str(file) + '.tsv', 'r', encoding="utf-8")
    for line in op:
        ou = line.strip().split('\t')
        sentence = ou[5].replace('\\n', ' ').replace('/', ' ').replace('*', ' ').replace('\\r', ' ').replace('\\t', ' ') + ' ' + ou[8]
    op.close()
 
    #names, l = chunking(sentence) 
    
    #preprocessing data deleting stop words, punctuations, ecc.  
    #final = preprocess(l) + names
    final = preprocess(word_tokenize(sentence))
    
    
    #CREATING INVERTED INDEX
    for word in final:
        index = vocabulary[word]
        if not (index in inverted_index):
            inverted_index[index] = ['doc_' + str(file)]
        elif not('doc_' + str(file) in inverted_index[index]):
            inverted_index[index] = inverted_index[index] + ['doc_' + str(file)]

op = open(r'C:\Users\mccol\Desktop\Sapienza\ADM\HW3\inverted_index.txt', 'w', encoding="utf-8")
op.write(str(inverted_index))
op.close()

In [32]:
#HANDLING USER INPUT
user_query = input()
l = []
names = []
for chunk in ne_chunk(pos_tag(word_tokenize(user_query))) : 
    if type(chunk) is nltk.tree.Tree:
        s = ''
        for  i in chunk:
            s = s + ' ' + i[0]
        names.append(s[1:])
    else:
        l.append(chunk[0])

final = []
    
#excluding stopwords and punctuation signs
for i in l:
    if not((ps.stem(i) in stopWords) or (ps.stem(i) in (string.punctuation) )):
        final.append(ps.stem(i))
final = final + names
final

garden pool lake


['garden', 'pool', 'lake']

In [33]:
voc = {}
inverted_idx = {}
i=0
for word in final: 
    voc[i]  = vocabulary[word]
    i = i+1
for index in range(i):
    inverted_idx[voc[index]] = inverted_index[voc[index]]

In [35]:
#finding list of docs that contain all the words in the query
docs = []

for i in range(18259):
    doc = 'doc_' + str(i)
    b = True
    for j in voc.values(): 
        b = b and (doc in inverted_idx[j])
    if b:
        docs.append(i)

df = f.filter(items = ['title', 'description', 'city', 'url']).loc[docs]
df.style.hide_index()